Python 3.9.7

Keras 2.9.0

First we import all libraries and packages we use.

In [ ]:
#Import .fit (Garmin) handling
from fitparse import FitFile

import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt 
#import ipynb.fit_functions as ffun
%run fit_functions.ipynb

#Signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

#Itertools
import itertools

#Time
import time


import json

#KERAS AND TENSORFLOW
import tensorflow as tf
tf.random.set_seed(1234)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
import keras
from keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

#TIMESERIES
from numpy.lib.stride_tricks import sliding_window_view

#GARBAGE COLLECTOR
import gc




The following function formats the input data correctly.
- dataset: data set
- sequence_length: number of back lagged values used for prediction
- sampling_rate: rate of samples in sequence_length
- n_forecast: number of points we want to predict

In [ ]:
def create_dataset(dataset, sequence_length,sampling_rate,n_forecast):
    dataX, dataY = [], []
    for i in range(len(dataset)-sequence_length*sampling_rate-n_forecast+1):
        a = dataset[i:(i+sequence_length*sampling_rate):sampling_rate]
        dataX.append(a)
        dataY.append(dataset[i + sequence_length*sampling_rate-(sampling_rate-1):i + sequence_length*sampling_rate+n_forecast-(sampling_rate-1)])
    return np.array(dataX), np.array(dataY)

The following functions calculate the HRV metrics.

In [ ]:
def interpolate(rr_manual):
    #Interpolate
    x = np.cumsum(rr_manual) #In seconds
    f = interp1d(x, rr_manual*1000, kind='cubic') #In milliseconds

    fs = 4.0 #Sample frequency
    steps = 1 / fs

    # now we can sample from interpolation function
    xx = np.arange(x[0], np.max(x), steps)
    rr_interpolated = f(xx)
    return rr_interpolated

def frequency_domain(rri, fs=4):
    # Estimate the spectral density using Welch's method
    fxx, pxx = signal.welch(x=rri, fs=fs)
    
    '''
    Segement found frequencies in the bands 
     - Very Low Frequency (VLF): 0-0.04Hz 
     - Low Frequency (LF): 0.04-0.15Hz 
     - High Frequency (HF): 0.15-0.4Hz
    '''
    cond_vlf = (fxx >= 0) & (fxx < 0.04)
    cond_lf = (fxx >= 0.04) & (fxx < 0.15)
    cond_hf = (fxx >= 0.15) & (fxx < 0.4)
    
    # calculate power in each band by integrating the spectral density 
    vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
    lf = trapz(pxx[cond_lf], fxx[cond_lf])
    hf = trapz(pxx[cond_hf], fxx[cond_hf])
    
    # sum these up to get total power
    total_power = vlf + lf + hf

    return vlf, lf, hf, total_power

def hrv_metric(rr):
    rr = np.array([rr])
    rr = rr.reshape(-1)

    # RMSSD: take the square root of the mean square of the differences
    rmssd = np.sqrt(np.mean(np.square(np.diff(rr*1000)))) #Need the data to be in milliseconds, therefore rr*1000

    # SDNN
    sdnn = np.std(rr*1000) #Need the data to be in milliseconds, therefore rr*1000

    try:
        rr_interpolated = interpolate(rr)

        vlf, lf, hf, total_power = frequency_domain(rr_interpolated,4)
    except:
        vlf, lf, hf, total_power = 'error','error','error','error'
        
    return rmssd, sdnn, vlf, lf, hf, total_power
    #return rmssd, sdnn

def cubic_spline(rr,start_hole,pred_length):

    time_rr = np.cumsum(rr)
    
    x1 = time_rr[start_hole-2]
    x2 = time_rr[start_hole-1]
    x3 = time_rr[start_hole+pred_length]
    x4 = time_rr[start_hole+pred_length+1]
    
    y1 = rr[start_hole-2]
    y2 = rr[start_hole-1]
    y3 = rr[start_hole+pred_length]
    y4 = rr[start_hole+pred_length]

    x = np.array([x1,x2,x3,x4])
    f = interp1d(x, [y1,y2,y3,y4], kind='cubic') #In milliseconds

    # now we can sample from interpolation function
    xx = np.linspace(x2, x3,pred_length+2)
    rr_interpolated = f(xx)
    return rr_interpolated

Import the Physionet data, specifically the 5 minute uphill walking segment.

In [ ]:
phys_path = 'ADD PATH HERE'
phys_files = ([f for f in listdir(phys_path) if isfile(join(phys_path, f)) 
                      and not f.startswith('.')]) #Include this to exclude .ds_store file
phys_files = np.sort(phys_files)
phys_files

phys_data = []
for part in phys_files:
    res_df = pd.read_csv(phys_path + part,sep=",")
    idx = res_df.loc[res_df['[Params]']=='[HRData]'].index[-1]
    idx+=1
    physionet = res_df.iloc[idx:-1].astype(int).values
    physionet = (physionet.reshape(-1))/1000
    phys_data.append(physionet)

physio_data = []
physio_data.append(phys_data[0][4822:5491]) 
physio_data.append(phys_data[1][3823:4661])
physio_data.append(phys_data[2][3798:4533])
physio_data.append(phys_data[3][2299:2997]) 
physio_data.append(phys_data[4][2634:3320]) 
physio_data.append(phys_data[6][1810:2485]) 
physio_data.append(phys_data[7][4075:4778]) 
physio_data.append(phys_data[8][2222:2886]) 
physio_data.append(phys_data[9][2543:3281]) 
physio_data.append(phys_data[10][2724:3482])
physio_data.append(phys_data[11][2211:2890]) 
physio_data.append(phys_data[12][3918:4671]) 


Create custom loss functions. 

In [ ]:
#CUSTOM LOSS FUNCTION
import keras.backend as K

#RMSSD
def rmssd_loss(y_true, y_pred):
    true = K.sqrt(K.mean(K.square(y_true[1:,:]-y_true[:-1,:])))
    pred = K.sqrt(K.mean(K.square(y_pred[1:,:]-y_pred[:-1,:])))
    loss = K.sqrt(K.mean(K.square(true-pred)))
    return loss

#DTW
class DtwLoss(keras.losses.Loss):
    def __init__(self, batch_size: int = 32):
        super(DtwLoss, self).__init__()
        self.batch_size = batch_size

    def call(self, y_true, y_pred):
        tmp = []
        for item in range(self.batch_size):
            s = y_true[item, :]
            t = y_pred[item, :]
            n, m = len(s), len(t)
            dtw_matrix = []
            for i in range(n + 1):
                line = []
                for j in range(m + 1):
                    if i == 0 and j == 0:
                        line.append(0)
                    else:
                        line.append(np.inf)
                dtw_matrix.append(line)

            for i in range(1, n + 1):
                for j in range(1, m + 1):
                    cost = tf.abs(s[i - 1] - t[j - 1])
                    last_min = tf.reduce_min([dtw_matrix[i - 1][j], dtw_matrix[i][j - 1], dtw_matrix[i - 1][j - 1]])
                    dtw_matrix[i][j] = tf.cast(cost, dtype=tf.float32) + tf.cast(last_min, dtype=tf.float32)

            temp = []
            for i in range(len(dtw_matrix)):
                temp.append(tf.stack(dtw_matrix[i]))

            tmp.append(tf.stack(temp)[n, m])
        return tf.reduce_mean(tmp)

Set scope of hyperparameter search in WandB.

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login()

#DEFINE SWEEP METHOD
sweep_config = {
    'method':'grid'
}

#DEFINE METRICS
metric = {
    'name':'val_loss',
    'goal':'minimize'
}

#DEFINE PARAMETERS
parameters_dict = {
    'number_layers': {
        'values':[2,4,6]
    },
    'units': {
        'values': [128,512,1024]
        },
    'sequence_length': {
        'values':[50]
    },
    'sampling_rate': {
        'values':[1,2]
    },
    'dropout': {
          'values': [0.0]
        },
    'batch_size':{
        'values': [1]
    },
    'learning_rate':{
        'values':[0.0015]
    },
    'epochs':{
        'values':[100]
    },
    'train_samples':{
        'values':[0.5]
    },
    'error_function':{
        'values':['RMSE','DTW','RMSSD']
    },
    'differencing':{
        'values':['YES','NO']
    },
    'network':{
        'values':['GRU','LSTM']
    },
    'len_test':{
        'values':[0.3]
    },
    'patience':{
        'values':[10]
    },
    'bidir':{
        'values':['YES','NO']
    },
    'n_forecast':{
        'values':[10]
    },
    'hole_length':{
        'values':[10]
    }
}



sweep_config['parameters'] = parameters_dict
sweep_config['metric'] = metric



Create the function that gets the predicted values and calculates all the HRV metrics and stores them in WandB.

In [ ]:
def predict(data_org, test, scaler, model_name, config,run, rolling):

    
    #GET PARAMETERS
    error_function = config['error_function']
    train_samples = config['train_samples']
    len_test = config['len_test']
    val_samples = 1-train_samples-len_test
    diff = config['differencing']
    n_forecast = config['n_forecast']
    hole_length = config['hole_length']
    sampling_rate=config["sampling_rate"]
    sequence_length=config["sequence_length"]
    batch_size = int(config["batch_size"]*len(data_org))


    #LOAD MODEL
    if error_function == 'RMSE':
        best_model = keras.models.load_model('article2/' + model_name  + '.h5',compile=False) #Need compile=False for it to work

    if error_function =='DTW':
        best_model = keras.models.load_model('article2/' + model_name  + '.h5',custom_objects={'DtwLoss':DtwLoss}, compile=False) #Need compile=False for it to work


    

    #GET ORIGINAL DATA WITH IMPUTED VALUES
    true_vals = test.copy()
    final_pred = true_vals.copy()
    start_hole = int(len(final_pred)/2)


    #IF DIFFERENCE
    if diff == 'YES':
        final_pred = np.diff(final_pred)
        start_hole = start_hole - 1 #Hole starts one before when differencing



    ###ROLLING PREDICTION###
    if rolling == 'YES':
        for i in range(n_forecast):
            
            #SCALE INPUT DATA
            input_data = final_pred.copy()
            input_data = input_data.reshape(len(final_pred),1)
            input_data = scaler.transform(input_data)
            input_data = input_data.reshape(len(final_pred))
            
            #PREPARE PREDICTION
            X_test,Y_test = create_dataset(input_data[:start_hole+i+1],sequence_length,sampling_rate,1)


            #PREDICT
            predictions = best_model.predict(X_test,verbose=0)

            
            #UNSCALE PREDICTION
            predictions = scaler.inverse_transform(predictions)
            predictions = predictions.reshape(len(predictions))
            
            #ADD PREDICTION TO final_pred
            final_pred[start_hole + i] = predictions[-1].copy() #Last value of prediction is the value we are after, since prepare_prediction is fed only up to this value


    ###MIX PREDICTION n_forecast MUST BE 2 HERE
    elif rolling=='MIX':
        for i in range(5):
            input_data = final_pred.reshape(-1,1).copy()
            input_data = scaler.transform(input_data)
            input_data = input_data.reshape(-1)
            X_test,Y_test = create_dataset(input_data[:start_hole+n_forecast + i*n_forecast],sequence_length,sampling_rate,n_forecast)



            predictions = best_model.predict(X_test,verbose=0)
            predictions = scaler.inverse_transform(predictions)
            predictions = predictions.reshape(-1)


            final_pred[start_hole+i*2:start_hole+n_forecast+i*n_forecast] = predictions[-n_forecast:].copy()

    
    
    ###SINGLE-SHOT PREDICTION###
    else:

        input_data = final_pred.reshape(-1,1).copy()
        input_data = scaler.transform(input_data)
        input_data = input_data.reshape(-1)
        X_test,Y_test = create_dataset(input_data[:start_hole+n_forecast],sequence_length,sampling_rate,n_forecast)


        predictions = best_model.predict(X_test,verbose=0)
        predictions = scaler.inverse_transform(predictions)
        predictions = predictions.reshape(-1)


        final_pred[start_hole:start_hole+n_forecast] = predictions[-n_forecast:]



    #FINAL PREDICTIONS IF THE DATA WERE DIFFERENCED
    if diff == 'YES':
        start_hole +=1
        preds = true_vals.copy()
        preds[1:start_hole+hole_length] = true_vals[0].copy() + np.cumsum(final_pred[:start_hole-1+hole_length]) #preds will start the hole one index before final_pred, since final_pred is differenced and preds is not.
        final_pred = preds.copy()

    
    #GET TRUE METRIC VALUES AND SAVE THEM IN WANDB
    true_rmssd, true_sdnn, true_vlf, true_lf, true_hf, true_total_power = hrv_metric(true_vals)
    wandb.run.summary['TRUE RMSSD'] = true_rmssd
    wandb.run.summary['TRUE SDNN'] = true_sdnn
    wandb.run.summary['TRUE VLF'] = true_vlf
    wandb.run.summary['TRUE LF'] = true_lf
    wandb.run.summary['TRUE HF'] = true_hf
    wandb.run.summary['TRUE TOTAL POWER'] = true_total_power

    #ROLLING PREDICTION SAVE IN WANDB
    if rolling == 'YES':
        #MAKE ARRAY OF PREDICTED VALUES
        pred_lengths = [1,3,5,7,10]
        for pred_length in pred_lengths:
            
            #GET ORIGINAL DATA WITH IMPUTED VALUES FOR VARIOUS HOLE SIZES (pred_length)
            rolling_prediction = true_vals.copy()
            rolling_prediction[:start_hole+pred_length] = final_pred[:start_hole+pred_length].copy()
            
            #GET METRICS
            rmssd, sdnn, vlf, lf, hf, total_power = hrv_metric(rolling_prediction)
        

            #SAVE METRICS IN WANDB
            hole_name = str(pred_length)
            wandb.run.summary[hole_name + 'RMSSD'] = rmssd
            wandb.run.summary[hole_name + 'SDNN'] = sdnn
            wandb.run.summary[hole_name + 'VLF'] = vlf
            wandb.run.summary[hole_name + 'LF'] = lf
            wandb.run.summary[hole_name + 'HF'] = hf
            wandb.run.summary[hole_name + 'TOTAL POWER'] = total_power

            #SAVE ERROR IN METRICS IN WANDB
            wandb.run.summary[hole_name + 'err_RMSSD'] = np.abs(rmssd-true_rmssd)/true_rmssd
            wandb.run.summary[hole_name + 'err_SDNN'] = np.abs(sdnn-true_sdnn)/true_sdnn
            wandb.run.summary[hole_name + 'err_VLF'] = np.abs(vlf-true_vlf)/true_vlf
            wandb.run.summary[hole_name + 'err_LF'] = np.abs(lf-true_lf)/true_lf
            wandb.run.summary[hole_name + 'err_HF'] = np.abs(hf-true_hf)/true_hf
            wandb.run.summary[hole_name + 'err_TOTAL POWER'] = np.abs(total_power-true_total_power)/true_total_power

            #CALCULATE CUBIC INTERPOLATION AND SAVE IN WANDB 
            rr_cubic = true_vals.copy()
            cubic_inter = cubic_spline(rr_cubic,start_hole,pred_length)
            rr_cubic[start_hole:start_hole+pred_length] = cubic_inter[1:-1].copy()
            cub_rmssd, cub_sdnn, cub_vlf, cub_lf, cub_hf, cub_total_power = hrv_metric(rr_cubic)
            
            wandb.run.summary[hole_name + 'CUB RMSSD'] = cub_rmssd
            wandb.run.summary[hole_name + 'CUB SDNN'] = cub_sdnn
            wandb.run.summary[hole_name + 'CUB VLF'] = cub_vlf
            wandb.run.summary[hole_name + 'CUB LF'] = cub_lf
            wandb.run.summary[hole_name + 'CUB HF'] = cub_hf
            wandb.run.summary[hole_name + 'CUB TOTAL POWER'] = cub_total_power
            wandb.run.summary[hole_name + 'cub_err_RMSSD'] = np.abs(cub_rmssd-true_rmssd)/true_rmssd
            wandb.run.summary[hole_name + 'cub_err_SDNN'] = np.abs(cub_sdnn-true_sdnn)/true_sdnn
            wandb.run.summary[hole_name + 'cub_err_VLF'] = np.abs(cub_vlf-true_vlf)/true_vlf
            wandb.run.summary[hole_name + 'cub_err_LF'] = np.abs(cub_lf-true_lf)/true_lf
            wandb.run.summary[hole_name + 'cub_err_HF'] = np.abs(cub_hf-true_hf)/true_hf
            wandb.run.summary[hole_name + 'cub_err_TOTAL POWER'] = np.abs(cub_total_power-true_total_power)/true_total_power

        

    else:
        #GET PREDICTED VALUES FOR METRICS
        rmssd, sdnn, vlf, lf, hf, total_power = hrv_metric(final_pred)
        hole_name = str(hole_length)
        wandb.run.summary[hole_name + 'RMSSD'] = rmssd
        wandb.run.summary[hole_name + 'SDNN'] = sdnn
        wandb.run.summary[hole_name + 'VLF'] = vlf
        wandb.run.summary[hole_name + 'LF'] = lf
        wandb.run.summary[hole_name + 'HF'] = hf
        wandb.run.summary[hole_name + 'TOTAL POWER'] = total_power



        wandb.run.summary[hole_name + 'err_RMSSD'] = np.abs(rmssd-true_rmssd)/true_rmssd
        wandb.run.summary[hole_name + 'err_SDNN'] = np.abs(sdnn-true_sdnn)/true_sdnn
        wandb.run.summary[hole_name + 'err_VLF'] = np.abs(vlf-true_vlf)/true_vlf
        wandb.run.summary[hole_name + 'err_LF'] = np.abs(lf-true_lf)/true_lf
        wandb.run.summary[hole_name + 'err_HF'] = np.abs(hf-true_hf)/true_hf
        wandb.run.summary[hole_name + 'err_TOTAL POWER'] = np.abs(total_power-true_total_power)/true_total_power




        #CALCULATE CUBIC INTERPOLATION AND SAVE IN WANDB 
        rr_cubic = true_vals.copy()
        cubic_inter = cubic_spline(rr_cubic,start_hole,hole_length)
        rr_cubic[start_hole:start_hole+hole_length] = cubic_inter[1:-1].copy()
        cub_rmssd, cub_sdnn, cub_vlf, cub_lf, cub_hf, cub_total_power = hrv_metric(rr_cubic)

        
        wandb.run.summary[hole_name + 'CUB RMSSD'] = cub_rmssd
        wandb.run.summary[hole_name + 'CUB SDNN'] = cub_sdnn
        wandb.run.summary[hole_name + 'CUB VLF'] = cub_vlf
        wandb.run.summary[hole_name + 'CUB LF'] = cub_lf
        wandb.run.summary[hole_name + 'CUB HF'] = cub_hf
        wandb.run.summary[hole_name + 'CUB TOTAL POWER'] = cub_total_power
        wandb.run.summary[hole_name + 'cub_err_RMSSD'] = np.abs(cub_rmssd-true_rmssd)/true_rmssd
        wandb.run.summary[hole_name + 'cub_err_SDNN'] = np.abs(cub_sdnn-true_sdnn)/true_sdnn
        wandb.run.summary[hole_name + 'cub_err_VLF'] = np.abs(cub_vlf-true_vlf)/true_vlf
        wandb.run.summary[hole_name + 'cub_err_LF'] = np.abs(cub_lf-true_lf)/true_lf
        wandb.run.summary[hole_name + 'cub_err_HF'] = np.abs(cub_hf-true_hf)/true_hf
        wandb.run.summary[hole_name + 'cub_err_TOTAL POWER'] = np.abs(cub_total_power-true_total_power)/true_total_power

        

    

    #SAVE ACTUAL AND PREDICTED VALUES AS WELL WHEN MIX CHANGE n_forecast TO 10
    wandb.run.summary['pred_vals'] = np.round(final_pred[start_hole:start_hole+hole_length].copy(), decimals=3)
    wandb.run.summary['true_vals'] = true_vals[start_hole:start_hole+hole_length].copy()
    rmse = np.sqrt(np.mean((final_pred[start_hole:start_hole+hole_length]-true_vals[start_hole:start_hole+hole_length])**2))
    wandb.run.summary['pred_RMSE'] = rmse



Build the model based on all the hyperparameter configurations.

In [ ]:
def train_function():

    #MARK DATA AS GLOBAL
    global target_data
    global person_nr

    #INITIALIZE RUN
    name = 'seq_' + 'person' + str(person_nr)
    run = wandb.init(name=name)
    config = wandb.config


    data = target_data.copy()
    data_org = target_data.copy()

    #DEFINE SWEEP PARAMETERS
    layers = config["number_layers"]
    units = config["units"]
    learning_rate = config["learning_rate"]
    dropout = config["dropout"]
    epochs = config["epochs"]

    train_samples = config["train_samples"]
    len_test = config['len_test']
    val_samples = 1-train_samples-len_test
    sampling_rate=config["sampling_rate"]
    sequence_length=config["sequence_length"]
    batch_size = int(config["batch_size"]*len(data))
    error_function = config["error_function"]
    diff = config["differencing"]
    network = config['network']
    patience = config['patience']


    #HOW MANY POINTS TO PREDICT
    n_forecast = config['n_forecast']

    #OPTIONS
    bidir = config['bidir'] #Must be NO when rolling is YES
    rolling = 'NO' # YES or NO or MIX

    #GET SET SIZES
    train_size = int(train_samples*len(data))
    val_size = int(val_samples*len(data))
    test = data[train_size+val_size:].copy()

    #DIFFERENCE
    if diff ==  'YES':
        data = np.diff(data)


    data = data.reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(data)
    data = scaler.transform(data)
    data = data.reshape(-1)

    #SPLIT DATA
    train = data[:train_size].copy()
    val = data[train_size:train_size+val_size].copy()



    inputs = keras.Input(shape=(sequence_length,1)) #First value in shape is length of each series, and second value is number of features
    x = inputs


    if bidir == 'YES':
        if network=='GRU':
            for layer in range(layers):
                x = keras.layers.Bidirectional(GRU(units=units,return_sequences=True))(x)
                x = keras.layers.Dropout(dropout)(x)
            x = keras.layers.Bidirectional(GRU(units=units,return_sequences=False))(x)
        else:
            for layer in range(layers):
                x = keras.layers.Bidirectional(LSTM(units=units,return_sequences=True))(x)
                x = keras.layers.Dropout(dropout)(x)
            x = keras.layers.Bidirectional(LSTM(units=units,return_sequences=False))(x)

    if bidir == 'NO':
        if network=='GRU':
            for layer in range(layers):
                x = keras.layers.GRU(units=units,return_sequences=True)(x)
                x = keras.layers.Dropout(dropout)(x)
            x = keras.layers.GRU(units=units,return_sequences=False)(x)
        else:
            for layer in range(layers):
                x = keras.layers.LSTM(units=units,return_sequences=True)(x)
                x = keras.layers.Dropout(dropout)(x)
            x = keras.layers.LSTM(units=units,return_sequences=False)(x)


    x = keras.layers.Dropout(dropout)(x)

    if rolling == 'YES':
        X_train, Y_train = create_dataset(train,sequence_length,sampling_rate,1)
        X_val, Y_val = create_dataset(val,sequence_length,sampling_rate,1)
        #train_dataset, val_dataset, test_dataset = prepare_data(train_samples, val_samples, sampling_rate, sequence_length, batch_size, data)
        print('We are rolling')
        outputs = keras.layers.Dense(1)(x) #With rolling prediction we only want one output (last step)

    else:
        X_train, Y_train = create_dataset(train,sequence_length,sampling_rate,n_forecast)
        X_val, Y_val = create_dataset(val,sequence_length,sampling_rate,n_forecast)
        #train_dataset, val_dataset, test_dataset = prepare_data2(train_samples, val_samples, sampling_rate, sequence_length, batch_size, data, n_forecast)
        print('Single-shot')
        outputs = keras.layers.Dense(1*n_forecast)(x) #When using single-shot output (multistep output), we need as many outputs as points we want to predict. If several features we need to multiply by features (I think)


    model = keras.Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    early_stop = keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience) #Stop when overfitting validation set, works well with 10. Stop when the validation loss is not improving after # epochs

    model_name = 'model'
    model_check = keras.callbacks.ModelCheckpoint('article2/' + model_name + '.h5',save_best_only=True, monitor="val_loss") #Save best model based on "monitor"
    callbacks = [model_check, early_stop,WandbCallback(save_model=False,save_graph=False)] #We dont need to save the models in wandb

    
    if error_function == 'RMSE':
        model.compile(optimizer=optimizer, loss='mean_squared_error')


    if error_function == 'DTW':
        #Batch_size must be 1 with DTW
        model.compile(optimizer=optimizer, loss=DtwLoss(batch_size=1))



    model.fit(X_train,Y_train,epochs=epochs, validation_data=(X_val,Y_val),callbacks=callbacks, shuffle=False, verbose=0)

    predict(data_org, test,scaler, model_name, config,run,rolling)


    #FINISH RUN
    run.finish()



BEFORE RUNNING:

Make sure that rolling='NO' when using Bidirectional layers

Run the code

In [ ]:
global person_nr
person_nr = 0

global target_data
target_data = physio_data[person_nr].copy() #Person 
    
sweep_id = wandb.sweep(sweep_config, project="PROJECT_NAME")
wandb.agent(sweep_id, function=train_function)